In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as PineconeStore
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
import os

c:\Users\NgLaam\Desktop\chatbot\medical-chatbot\medical-chatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY = '8d7fd4ae-327c-4cd5-abbe-0c2d4769e37e'

In [3]:
# Thiết lập biến môi trường cho Pinecone API key
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [5]:
# #Extract PDF data
# data_path = 'C:/Users/NgLaam/Desktop/chatbot/medical-chatbot/dataset'

# def load_pdf(data_path):
#     loader = DirectoryLoader(data_path,
#                         glob = '*.pdf',
#                         loader_cls=PyPDFLoader)
    
#     documents = loader.load()

#     return documents

# extract_data = load_pdf(data_path)

In [11]:
# extract_data

In [6]:
# #Create text chunks
# def text_split(extract_data):
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size = 512, chunk_overlap = 20)
#     text_chunks = text_splitter.split_documents(extract_data)

#     return text_chunks

# text_chunks = text_split(extract_data)
# print('Length of my chunk is ', len(text_chunks))

In [4]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embeddings = download_hugging_face_embeddings()

In [7]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [40]:
query = embeddings.embed_query('Xin chào Việt Nam')
print('Length', len(query))

Length 384


In [5]:
pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)

index_name ="vni-medical"

# #Creating Embeddings for each of the text chunks and storing 
# docsearch = PineconeStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [46]:
# prompt_template="""
# Use the following pieces of information to answer the user's question.
# If you don't know the answer, just say that you don't know, don't try to make up an answer.

# Context: {context}
# Question: {question}

# Only return the helpful answer below and nothing else.
# Helpful answer:
# """

In [47]:
# PROMPT = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])
# chain_type_kwargs={'prompt': PROMPT}

In [52]:
# llm=CTransformers(model='C:/Users/NgLaam/Desktop/chatbot/medical-chatbot/model/llama-2-7b-chat.ggmlv3.q4_0.bin',
#                   model_type='llama',
#                   config={'max_new_tokens': 512,
#                           'temperature':0.8})

In [53]:
# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type='stuff',
#     retriever = docsearch.as_retriever(search_kwargs={'k': 2}),
#     return_source_documents=True,
#     chain_type_kwargs=chain_type_kwargs
# )

In [2]:
model_path = "vinai/PhoGPT-4B-Chat"

config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
config.init_device = "cpu"
# config.attn_config['attn_impl'] = 'flash' # If installed: this will use either Flash Attention V1 or V2 depending on what is installed

C:\Users\NgLaam\.cache\huggingface\modules\transformers_modules\vinai\PhoGPT-4B-Chat\116013fa63f8c4025739487e1cbff65b7375bbe2\configuration_mpt.py:114: UserWarning: alibi or rope is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi or rope is turned on, setting `learned_pos_emb` to `False.`')
C:\Users\NgLaam\.cache\huggingface\modules\transformers_modules\vinai\PhoGPT-4B-Chat\116013fa63f8c4025739487e1cbff65b7375bbe2\configuration_mpt.py:141: UserWarning: If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".
  warnings.warn(UserWarning('If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".'))


In [ ]:
cache_directory = "C:/Users/NgLaam/Desktop/chatbot/medical-chatbot/model/vietnamese7b-llama"

model = AutoModelForCausalLM.from_pretrained(model_path, config=config, torch_dtype=torch.bfloat16, trust_remote_code=True, cache_dir=cache_directory)
# If your GPU does not support bfloat16:
# model = AutoModelForCausalLM.from_pretrained(model_path, config=config, torch_dtype=torch.float16, trust_remote_code=True)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True, cache_dir=cache_directory)

In [3]:
load_dir = 'C:/Users/NgLaam/Desktop/chatbot/medical-chatbot/model/vietnamese7b-llama/models--vinai--PhoGPT-4B-Chat/snapshots/116013fa63f8c4025739487e1cbff65b7375bbe2'
model = AutoModelForCausalLM.from_pretrained(load_dir, config=config, torch_dtype=torch.bfloat16, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(load_dir, trust_remote_code=True)

In [15]:
# import re

# docsearch = PineconeStore.from_existing_index(index_name, embeddings)

# query = 'sử dụng thuốc chống sốt rét nên lưu ý điều gì'
# similar = docsearch.similarity_search(query, k=3)

# # Use a different variable name instead of str
# str_similar = str(similar[1])

# def clean_text(text):
#     # Remove 'page_content=' from the string
#     result_str = re.sub(r"page_content='|\\n|'", "", text)
#     return result_str

# print(clean_text(str_similar))


rét,  chúng  nên  được  dùng  vào  cùng  một  thời  điểm  mỗi  ngày.Thuốc  chống  sốt  rét  chỉ  được  cung  cấp  khi  có  đơn  thuốc  của  bác  sĩ.  Chúng  có  dạng  viên,  viên  nang  và  dạng  tiêm.  Trong  số  các  thuốc  chống  sốt  rét  thường  được  sử  dụng286 BẠC  THÀNH  Y  HỌC  GALE  2Machine Translated by Google


In [16]:
# #Load index if we already have index

# def get_answer(docsearch, query, model, max_length = 512):
#     concentrated_docs = ''
#     docs = docsearch.similarity_search(query, k=2)
#     for doc in docs:
#         clean_doc  = clean_text(str(doc))
#         concentrated_docs += clean_doc
    
#     print(concentrated_docs)
#     input_ids = tokenizer.encode(concentrated_docs, return_tensors='pt')
#     sample_outputs = model.generate(input_ids,pad_token_id=tokenizer.eos_token_id,
#                                    do_sample=True,
#                                    max_length=max_length,
#                                    min_length=max_length,
#                                    top_k=40,
#                                    num_beams=5,
#                                    early_stopping=True,
#                                    no_repeat_ngram_size=2,
#                                    num_return_sequences=1)

#     generated_sequences = [tokenizer.decode(output, skip_special_tokens=True) for output in sample_outputs]
    
#     return list(enumerate(generated_sequences))

In [19]:
# query = 'Dị ứng là gì'
# result = get_answer(docsearch, query, model)
# print(result)

Trị  liệu  hành  vi  xem  Trị  liệu  nhận  thức  hành  viNhiễm  sán  dây  bò  xem  bệnh  sán  dâySự  định  nghĩaSự  đối  đãiSự  miêu  tảhội  chứng  BehcetKHÁCBẠC  THÀNH  Y  HỌC  GALE  2TỔ  CHỨCSÁCH“Lở  loét  trên  giường.”  Bộ  Y  tế  và  Thảo  dược  Hoa  Kỳ.  Ngày  15  tháng  3  năm  1998  <http://www.healthherbs.com>.Ban  cố  vấn  loét  áp  lực  quốc  gia.  SUNY  tại  Buffalo,“Phá  vỡ  kết  nối  giữa  OR  và  loét  áp  lực.”bao  quanh  quầng  vú  (vùng  sẫm  màu  xung  quanh  núm  vú)  và  kéo  dài  xuống  phía  dưới  và  xung  quanh  mặt  dưới  của  vú.  Điều  này  tạo  ra  vết  sẹo  ít  dễ  thấy  nhất.  Các  mô,  mỡ  và  da  thừa  sẽ  được  loại  bỏ,  đồng  thời  núm  vú  và  quầng  vú  được  đặt  lại  vị  trí.Trong  một  số  trường  hợp  nhất  định,  hút  mỡ  (hút  mỡ)  được  sử  dụng  đểNhững  nam  giới  có  bộ  ngực  to  (gynecomastia)  cũng  có  thể  là  ứng  cử
[(0, 'Trị  liệu  hành  vi  xem  Trị  liệu  nhận  thức  hành  viNhiễm  sán  dây  bò  xem  bệnh  sán  dâySự  định  nghĩ

In [30]:
# while True:
#     user_input=input(f'Input Prompt:')
#     result=get_answer(docsearch, query, model)
#     print('Response :', result)

có  thể  đề  nghị  xét  nghiệm  máu  để  đo  nồng  độ  hormone,nội  tiết  tố  nam,  có  thể  giúp  giảm  lông  trên  cơ  thể.  Nếu  mộtcăng  thẳng  là  hai  nguyên  nhân  chính.  Mất  kinh  thường  làkhối  u.  Việc  chấm  dứt  kinh  nguyệt  cũng  xảy  ra  vĩnh  viễn  sau  khi  mãn  kinh  hoặc  cắt  bỏ  tử  cung.nhưng  việc  khám  nên  bắt  đầu  bằng  que  thử  thai;  khả  năng  mang  thaicó  kinh  nguyệt  bình  thường  trước  đây.  Nhiều  phụ  nữ  thỉnh  thoảng  bị  mất  kinh  mỗi  tháng.  Vô  kinh  xảy  ra  nếu  người  phụ  nữ  mất  kinh  từ  ba  kỳ  kinh  trở  lên  liên  tiếp.Việc  không  có  kinh  nguyệt  được  gọi  là  amenor-rhea.  Vô  kinh  nguyên  phát  là  tình  trạng  không  bắt  đầu  có  kinhRebecca  J.  FreyVô  kinh  có  thể  có  nhiều  nguyên  nhân.  Vô  kinh  nguyên  phát  có  thể  là  kết  quả  của  sự  mất  cân  bằng  nội  tiết  tố,  rối  loạn  tâm  thần,  rối
Response : [(0, 'có  thể  đề  nghị  xét  nghiệm  máu  để  đo  nồng  độ  hormone,nội  tiết  tố  nam,  có  thể  gi

In [9]:
PROMPT_TEMPLATE = "### Câu hỏi: {instruction}\n### Trả lời:"

# Some instruction examples
# instruction = "Viết bài văn nghị luận xã hội về {topic}"
# instruction = "Viết bản mô tả công việc cho vị trí {job_title}"
# instruction = "Sửa lỗi chính tả:\n{sentence_or_paragraph}"
# instruction = "Dựa vào văn bản sau đây:\n{text}\nHãy trả lời câu hỏi: {question}"
# instruction = "Tóm tắt văn bản:\n{text}"

instruction = "Sử dụng mẩu thông tin sau và kiến thức của bạn :\nDị ứng mẩn đỏ và tôi biết nó là ai tôi xin không biết\n Hãy trả lời câu hỏi: Dị ứng gây ra gì"
# instruction = "Sửa lỗi chính tả:\nTriệt phá băng nhóm kướp ô tô, sử dụng \"vũ khí nóng\""

input_prompt = PROMPT_TEMPLATE.format_map({"instruction": instruction})

input_ids = tokenizer(input_prompt, return_tensors="pt")

# Move model to CUDA device
model.to("cpu")

MPTForCausalLM(
  (transformer): MPTModel(
    (wte): SharedEmbedding(20480, 3072)
    (emb_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-31): 32 x MPTBlock(
        (norm_1): LPLayerNorm((3072,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (Wqkv): Linear(in_features=3072, out_features=9216, bias=True)
          (out_proj): Linear(in_features=3072, out_features=3072, bias=True)
        )
        (norm_2): LPLayerNorm((3072,), eps=1e-05, elementwise_affine=True)
        (ffn): MPTMLP(
          (up_proj): Linear(in_features=3072, out_features=12288, bias=True)
          (down_proj): Linear(in_features=12288, out_features=3072, bias=True)
        )
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_ffn_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm_f): LPLayerNorm((3072,), eps=1e-05, elementwise_affine=True)
  )
)

In [10]:
outputs = model.generate(
    inputs=input_ids["input_ids"].to("cpu"),
    attention_mask=input_ids["attention_mask"].to("cpu"),
    do_sample=True,
    temperature=1.0,
    top_k=50,
    top_p=0.9,
    max_new_tokens=512,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id
)

response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
response = response.split("### Trả lời:")[1]
response

C:\Users\NgLaam\.cache\huggingface\modules\transformers_modules\vinai\PhoGPT-4B-Chat\116013fa63f8c4025739487e1cbff65b7375bbe2\attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagating key_padding_mask to the attention module ' + 'and applying it within the attention module can cause ' + 'unnecessary computation/memory usage. Consider integrating ' + 'into attn_bias once and passing that to each attention ' + 'module instead.')


KeyboardInterrupt: 